In [3]:
import pandas as pd
import sys 
sys.path.append("../")

In [4]:
import torch
torch.cuda.device_count()

1

In [8]:
from CausalBert_base import CausalBertWrapper
df = pd.read_csv("../testdata.csv")

cb = CausalBertWrapper(batch_size=2, g_weight=0.1, Q_weight=0.1,mlm_weight=1)
cb.train(df["text"], df["C"], df["T"], df["Y"], epochs = 3)

probs, preds, Ys = cb.inference(df["text"], df["C"])

print(cb.ATE(df['C'],df['text'],platt_scaling=False))

Some weights of CausalBert were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['Q_cls.0.0.bias', 'Q_cls.0.0.weight', 'Q_cls.0.2.bias', 'Q_cls.0.2.weight', 'Q_cls.1.0.bias', 'Q_cls.1.0.weight', 'Q_cls.1.2.bias', 'Q_cls.1.2.weight', 'g_cls.bias', 'g_cls.weight', 'vocab_projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/.venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/root/.venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be re

-0.22335845758647677


In [10]:
import numpy as np
ind = (preds == 1)
np.mean(probs[ind][:,0] - probs[ind][:,1])

-0.21266833189349585

In [6]:
preds

array([1, 1, 1, ..., 1, 1, 1])

In [10]:
cb.model

CausalBert(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear

In [9]:
probs

array([[0.31508371, 0.55072117],
       [0.31851456, 0.5608657 ],
       [0.31563544, 0.56917328],
       ...,
       [0.33964068, 0.55238307],
       [0.34313986, 0.54485083],
       [0.30500376, 0.56488234]])

In [5]:
print((preds == Ys).sum() / len(preds))

0.0


In [8]:
from collections import defaultdict
import numpy as np
def ATE_unadjusted(T, Y):
    x = defaultdict(list)
    for t, y in zip(T, Y):
        x[t].append(y)
    T0 = np.mean(x[0])
    T1 = np.mean(x[1])
    return T0 - T1

def ATE_adjusted(C, T, Y):
    x = defaultdict(list)
    for c, t, y in zip(C, T, Y):
        x[c, t].append(y)

    C0_ATE = np.mean(x[0,0]) - np.mean(x[0,1])
    C1_ATE = np.mean(x[1,0]) - np.mean(x[1,1])
    print(C0_ATE, C1_ATE)
    print(x)
    return np.mean([C0_ATE, C1_ATE])


print("ATE_unadjusted: ", ATE_unadjusted(df["T"], df["Y"]))
print("ATE_adjusted: ", ATE_adjusted(df["C"], df["T"],df["Y"]))

ATE_unadjusted:  -0.2240035541871202
-0.1582778211984921 -0.17131718637675425
defaultdict(<class 'list'>, {(1, 1): [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,

In [9]:
df.head()

,Unnamed: 0,text,T,C,Y
0,0,this is a great cd full of worship favorites!!...,1,1,0
1,1,keith green had his special comedy style of ch...,1,1,1
2,2,keith green was a true gift of god. his music ...,1,0,0
3,3,keith's music is a timeless message. since hi...,1,0,0
4,4,"if you're looking for a meditative, contemplat...",1,0,1


In [ ]:
from CausalBert import CausalBertWrapper
df = pd.read_csv("../testdata.csv")

cb = CausalBertWrapper(batch_size=2, g_weight=0.1, Q_weight=0.1,mlm_weight=1)
cb.train(df["text"], df["C"], df["T"], df["Y"], epochs = 1)
print(cb.ATE(df['C'],df['text'],platt_scaling=True))

In [13]:
len(df["T"])

18403